In [1]:
from pathlib import Path
from typing import Annotated, Union

import typer
from peft import AutoPeftModelForCausalLM, PeftModelForCausalLM
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    PreTrainedModel,
    PreTrainedTokenizer,
    PreTrainedTokenizerFast,
)

/home/reymond/anaconda3/envs/afac2024/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
ModelType = Union[PreTrainedModel, PeftModelForCausalLM]
TokenizerType = Union[PreTrainedTokenizer, PreTrainedTokenizerFast]

In [3]:
def _resolve_path(path: Union[str, Path]) -> Path:
    return Path(path).expanduser().resolve()


def load_model_and_tokenizer(model_dir: Union[str, Path]) -> tuple[ModelType, TokenizerType]:
    model_dir = _resolve_path(model_dir)
    if (model_dir / 'adapter_config.json').exists():
        model = AutoPeftModelForCausalLM.from_pretrained(
            model_dir, trust_remote_code=True, device_map='auto'
        )
        tokenizer_dir = model.peft_config['default'].base_model_name_or_path
    else:
        model = AutoModelForCausalLM.from_pretrained(
            model_dir, trust_remote_code=True, device_map='auto'
        )
        tokenizer_dir = model_dir
    tokenizer = AutoTokenizer.from_pretrained(
        tokenizer_dir, trust_remote_code=True
    )
    return model, tokenizer

In [4]:
model_dir = "output/checkpoint-1000"

model, tokenizer = load_model_and_tokenizer(model_dir)

Loading checkpoint shards: 100%|██████████| 7/7 [02:54<00:00, 24.98s/it]
Setting eos_token is not supported, use the default one.
Setting pad_token is not supported, use the default one.
Setting unk_token is not supported, use the default one.


In [5]:
import pandas as pd
from tools.common_utils import highlight_diff, read_jsonl

json_file_path = 'data/dev_api.json'
data_list = read_jsonl(json_file_path)

df = pd.DataFrame(data_list)
df.head()

,conversations
0,"[{'role': 'user', 'content': '你现在是一个金融领域专家，你需要..."
1,"[{'role': 'user', 'content': '你现在是一个金融领域专家，你需要..."
2,"[{'role': 'user', 'content': '你现在是一个金融领域专家，你需要..."
3,"[{'role': 'user', 'content': '你现在是一个金融领域专家，你需要..."
4,"[{'role': 'user', 'content': '你现在是一个金融领域专家，你需要..."


In [7]:
from random import random

top_p=0.7 
temperature=0.95

total_eval_count = 0.0
correct_count = 0.0

for index, row in df.iterrows():
    if random() <= 0.9:
        continue
    
    single_user_row = row["conversations"][0]
    single_assis_row = row["conversations"][1]
    
    input = single_user_row["content"]
    output = single_assis_row["content"]
    
    
    response, history = model.chat(tokenizer=tokenizer, query=input, history=[], top_p=top_p, temperature=temperature)
    
    # optimized_resp = optimize_parameters(response, fund_standard_name, stock_standard_name)
    
    total_eval_count += 1
    if response == output:
        correct_count += 1
    else:
        print("-----data index-----")
        print(index)
        print("-----query input-----")
        print(input)
        print("-----output diff-----")
        print(highlight_diff(output, response))
        print(response)
        print()
    
    if total_eval_count == 20:
        break
    
print("预测正确的比例：" + f"{correct_count / total_eval_count :.2%}")

-----data index-----
42
-----query input-----
你现在是一个金融领域专家，你需要通过编排api来得到用户query的答案，输出json格式。query是：张江ETF现在的单位净值和近1个月的收益率同类排名情况如何？ 
 query中使用到的api可能是：条件选基_查询近期收益率同类排名、基金查询_查询近期收益率同类排名、条件选基_查询近期年化收益率同类排名、基金查询_查询近期年化收益率同类排名、条件选基_查询近期波动率同类排名、基金查询_查询近期波动率同类排名、条件选基_查询近期夏普比率同类排名、条件选基_查询近期卡玛比率同类排名、条件选基_查询近期信息比率同类排名、基金查询_查询近期夏普比率同类排名、基金查询_查询近期卡玛比率同类排名、基金查询_查询近期信息比率同类排名、条件选基_查询单位净值同步日期、基金查询_查询单位净值同步日期、条件选基_查询近期最大回撤同类排名、基金查询_查询近期最大回撤同类排名、条件选基_查询单位净值、基金查询_查询单位净值、条件选基_查询近期收益率、基金查询_查询近期收益率、股票查询_查询净资产收益率、条件选股_查询净资产收益率、条件选基_查询近期年化收益率、基金查询_查询近期年化收益率、条件选基_查询近期超大盘收益率、基金查询_查询近期超大盘收益率、条件选基_查询近一年买入持有6个月历史盈利概率、基金查询_查询近一年买入持有6个月历史盈利概率、条件选基_查询年度收益率、基金查询_查询年度收益率、条件选基_查询近5年买入持有1年历史盈利概率、条件选基_查询近3年买入持有1年历史盈利概率、基金查询_查询近5年买入持有1年历史盈利概率、基金查询_查询近3年买入持有1年历史盈利概率、股票查询_查询净利率、条件选股_查询净利率、股票查询_查询每股收益、条件选股_查询每股收益、条件选基_查询近期夏普率、基金查询_查询近期夏普率、股票查询_查询营收同比增长、条件选股_查询营收同比增长、股票查询_查询每股收益ttm、股票查询_查询净利润同比增长、条件选股_查询每股收益ttm、条件选股_查询净利润同比增长、条件选基_查询每十份收益单位派息（元）、基金查询_查询每十份收益单位派息（元）、条件选基_查询类型、基金查询_查询类型、数值计算_加法计算、数值计算_减法计算、数值计算_乘法计算、数值计算_除法计算、逻辑计